# W_LEDGER_D Dimension ETL Process
### EDW Staging (W_LEDGER_DS) to Dimension (W_LEDGER_D) - SCD Type 1

In [ ]:
%sql
-- 1) PARAMETERS: Derive ETL control parameters
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT
  CASE 
    WHEN COUNT(*) > 0 THEN 'Y'
    ELSE 'N'
  END AS IS_INCREMENTAL,
  380 AS DATASOURCE_NUM_ID,
  '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
  '1' AS TENANT_ID,
  2624982 AS ETL_PROC_WID,
  12345 AS EXECUTION_ID,
  30 AS PRUNE_DAYS
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SILOS_SIL_LEDGERDIMENSION'
  AND DATASOURCE_NUM_ID = 380
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'
  AND COMMITTED = '1';

In [ ]:
%sql
-- 2) LOOKUP STAGING: W_USER_D for CREATED_BY lookup
CREATE OR REPLACE TEMP VIEW lkp_user_created AS
SELECT
  DATASOURCE_NUM_ID,
  ROW_WID,
  INTEGRATION_ID,
  EFFECTIVE_TO_DT,
  EFFECTIVE_FROM_DT
FROM workspace.oracle_edw.w_user_d
WHERE DELETE_FLG = 'N';

In [ ]:
%sql
-- 3) LOOKUP STAGING: W_USER_D for CHANGED_BY lookup
CREATE OR REPLACE TEMP VIEW lkp_user_changed AS
SELECT
  DATASOURCE_NUM_ID,
  ROW_WID,
  INTEGRATION_ID,
  EFFECTIVE_TO_DT,
  EFFECTIVE_FROM_DT
FROM workspace.oracle_edw.w_user_d
WHERE DELETE_FLG = 'N';

In [ ]:
%sql
-- 4) SOURCE STAGING: Join staging table with user lookups
CREATE OR REPLACE TEMP VIEW stg_ledger_with_lookups AS
SELECT
  ds.LEDGER_NAME,
  ds.LEDGER_SHORT_NAME,
  ds.LEDGER_DESC,
  ds.CHART_OF_ACCOUNTS,
  ds.CURRENCY_CODE,
  ds.CALENDER_NAME,
  ds.SLA_ACCOUNTING_METHOD_CODE,
  ds.PERIOD_TYPE,
  ds.LEDGER_CATEGORY_CODE,
  ds.SET_ID,
  ds.CREATED_BY_ID,
  ds.CHANGED_BY_ID,
  ds.CREATED_ON_DT,
  ds.CHANGED_ON_DT,
  ds.AUX1_CHANGED_ON_DT,
  ds.AUX2_CHANGED_ON_DT,
  ds.AUX3_CHANGED_ON_DT,
  ds.AUX4_CHANGED_ON_DT,
  ds.DELETE_FLG,
  ds.DATASOURCE_NUM_ID,
  ds.INTEGRATION_ID,
  ds.TENANT_ID,
  ds.X_CUSTOM,
  lkp_created.ROW_WID AS CREATED_BY_ROW_WID,
  lkp_changed.ROW_WID AS CHANGED_BY_ROW_WID
FROM workspace.oracle_edw.w_ledger_ds ds
LEFT JOIN lkp_user_created lkp_created
  ON ds.DATASOURCE_NUM_ID = lkp_created.DATASOURCE_NUM_ID
  AND ds.CREATED_BY_ID = lkp_created.INTEGRATION_ID
  AND ds.CREATED_ON_DT >= lkp_created.EFFECTIVE_FROM_DT
  AND ds.CREATED_ON_DT < lkp_created.EFFECTIVE_TO_DT
LEFT JOIN lkp_user_changed lkp_changed
  ON ds.DATASOURCE_NUM_ID = lkp_changed.DATASOURCE_NUM_ID
  AND ds.CHANGED_BY_ID = lkp_changed.INTEGRATION_ID
  AND ds.CHANGED_ON_DT >= lkp_changed.EFFECTIVE_FROM_DT
  AND ds.CHANGED_ON_DT < lkp_changed.EFFECTIVE_TO_DT;

In [ ]:
%sql
-- 5) TRANSFORMATION: Apply business rules and prepare for load
CREATE OR REPLACE TEMP VIEW stg_ledger_transformed AS
SELECT
  LEDGER_NAME,
  LEDGER_SHORT_NAME,
  LEDGER_DESC,
  COALESCE(CHART_OF_ACCOUNTS, '__NOT_APPLICABLE__') AS CHART_OF_ACCOUNTS,
  COALESCE(CURRENCY_CODE, '__NOT_APPLICABLE__') AS CURRENCY_CODE,
  CALENDER_NAME,
  COALESCE(SLA_ACCOUNTING_METHOD_CODE, '__NOT_APPLICABLE__') AS SLA_ACCOUNTING_METHOD_CODE,
  PERIOD_TYPE,
  COALESCE(LEDGER_CATEGORY_CODE, '__NOT_APPLICABLE__') AS LEDGER_CATEGORY_CODE,
  SET_ID,
  COALESCE(CREATED_BY_ROW_WID, 0) AS CREATED_BY_WID,
  COALESCE(CHANGED_BY_ROW_WID, 0) AS CHANGED_BY_WID,
  CREATED_ON_DT,
  CHANGED_ON_DT,
  AUX1_CHANGED_ON_DT,
  AUX2_CHANGED_ON_DT,
  AUX3_CHANGED_ON_DT,
  AUX4_CHANGED_ON_DT,
  CASE WHEN DELETE_FLG = 'Y' THEN 'Y' ELSE 'N' END AS DELETE_FLG,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  TENANT_ID,
  X_CUSTOM
FROM stg_ledger_with_lookups;

In [ ]:
%sql
-- 6) CHANGE DETECTION: Determine Insert/Update/No-Action (IND_UPDATE)
CREATE OR REPLACE TEMP VIEW stg_ledger_with_action AS
SELECT
  s.*,
  CASE
    -- No change: Record exists and all change tracking dates match
    WHEN t.ROW_WID IS NOT NULL
      AND (t.CHANGED_ON_DT = s.CHANGED_ON_DT OR (t.CHANGED_ON_DT IS NULL AND s.CHANGED_ON_DT IS NULL))
      AND (t.AUX1_CHANGED_ON_DT = s.AUX1_CHANGED_ON_DT OR (t.AUX1_CHANGED_ON_DT IS NULL AND s.AUX1_CHANGED_ON_DT IS NULL))
      AND (t.AUX2_CHANGED_ON_DT = s.AUX2_CHANGED_ON_DT OR (t.AUX2_CHANGED_ON_DT IS NULL AND s.AUX2_CHANGED_ON_DT IS NULL))
      AND (t.AUX3_CHANGED_ON_DT = s.AUX3_CHANGED_ON_DT OR (t.AUX3_CHANGED_ON_DT IS NULL AND s.AUX3_CHANGED_ON_DT IS NULL))
      AND (t.AUX4_CHANGED_ON_DT = s.AUX4_CHANGED_ON_DT OR (t.AUX4_CHANGED_ON_DT IS NULL AND s.AUX4_CHANGED_ON_DT IS NULL))
    THEN 'N'
    -- Update: Record exists but has changes
    WHEN t.ROW_WID IS NOT NULL THEN 'U'
    -- Insert: New record
    ELSE 'I'
  END AS IND_UPDATE,
  t.ROW_WID AS EXISTING_ROW_WID
FROM stg_ledger_transformed s
LEFT JOIN workspace.oracle_edw.w_ledger_d t
  ON s.DATASOURCE_NUM_ID = t.DATASOURCE_NUM_ID
  AND s.INTEGRATION_ID = t.INTEGRATION_ID;

In [ ]:
%sql
-- 7) MERGE: Upsert data into target dimension table
MERGE INTO workspace.oracle_edw.w_ledger_d AS tgt
USING (
  SELECT
    s.LEDGER_NAME,
    s.LEDGER_SHORT_NAME,
    s.LEDGER_DESC,
    s.CHART_OF_ACCOUNTS,
    s.CURRENCY_CODE,
    s.CALENDER_NAME,
    s.SLA_ACCOUNTING_METHOD_CODE,
    s.PERIOD_TYPE,
    s.LEDGER_CATEGORY_CODE,
    s.SET_ID,
    s.CREATED_BY_WID,
    s.CHANGED_BY_WID,
    s.CREATED_ON_DT,
    s.CHANGED_ON_DT,
    s.AUX1_CHANGED_ON_DT,
    s.AUX2_CHANGED_ON_DT,
    s.AUX3_CHANGED_ON_DT,
    s.AUX4_CHANGED_ON_DT,
    s.DELETE_FLG,
    s.DATASOURCE_NUM_ID,
    s.INTEGRATION_ID,
    s.TENANT_ID,
    s.X_CUSTOM,
    s.IND_UPDATE,
    p.ETL_PROC_WID,
    current_timestamp() AS CURRENT_TS
  FROM stg_ledger_with_action s
  CROSS JOIN etl_params p
  WHERE s.IND_UPDATE IN ('I', 'U')  -- Only process inserts and updates
) src
ON tgt.DATASOURCE_NUM_ID = src.DATASOURCE_NUM_ID
  AND tgt.INTEGRATION_ID = src.INTEGRATION_ID
WHEN MATCHED AND src.IND_UPDATE = 'U' THEN
  UPDATE SET
    tgt.LEDGER_NAME = src.LEDGER_NAME,
    tgt.LEDGER_SHORT_NAME = src.LEDGER_SHORT_NAME,
    tgt.LEDGER_DESC = src.LEDGER_DESC,
    tgt.CHART_OF_ACCOUNTS = src.CHART_OF_ACCOUNTS,
    tgt.CURRENCY_CODE = src.CURRENCY_CODE,
    tgt.CALENDER_NAME = src.CALENDER_NAME,
    tgt.SLA_ACCOUNTING_METHOD_CODE = src.SLA_ACCOUNTING_METHOD_CODE,
    tgt.PERIOD_TYPE = src.PERIOD_TYPE,
    tgt.LEDGER_CATEGORY_CODE = src.LEDGER_CATEGORY_CODE,
    tgt.SET_ID = src.SET_ID,
    tgt.CREATED_BY_WID = src.CREATED_BY_WID,
    tgt.CHANGED_BY_WID = src.CHANGED_BY_WID,
    tgt.CREATED_ON_DT = src.CREATED_ON_DT,
    tgt.CHANGED_ON_DT = src.CHANGED_ON_DT,
    tgt.AUX1_CHANGED_ON_DT = src.AUX1_CHANGED_ON_DT,
    tgt.AUX2_CHANGED_ON_DT = src.AUX2_CHANGED_ON_DT,
    tgt.AUX3_CHANGED_ON_DT = src.AUX3_CHANGED_ON_DT,
    tgt.AUX4_CHANGED_ON_DT = src.AUX4_CHANGED_ON_DT,
    tgt.DELETE_FLG = src.DELETE_FLG,
    tgt.TENANT_ID = src.TENANT_ID,
    tgt.X_CUSTOM = src.X_CUSTOM,
    tgt.W_UPDATE_DT = src.CURRENT_TS,
    tgt.ETL_PROC_WID = src.ETL_PROC_WID
WHEN NOT MATCHED AND src.IND_UPDATE = 'I' THEN
  INSERT (
    LEDGER_NAME,
    LEDGER_SHORT_NAME,
    LEDGER_DESC,
    CHART_OF_ACCOUNTS,
    CURRENCY_CODE,
    CALENDER_NAME,
    SLA_ACCOUNTING_METHOD_CODE,
    PERIOD_TYPE,
    LEDGER_CATEGORY_CODE,
    SET_ID,
    CREATED_BY_WID,
    CHANGED_BY_WID,
    CREATED_ON_DT,
    CHANGED_ON_DT,
    AUX1_CHANGED_ON_DT,
    AUX2_CHANGED_ON_DT,
    AUX3_CHANGED_ON_DT,
    AUX4_CHANGED_ON_DT,
    DELETE_FLG,
    DATASOURCE_NUM_ID,
    INTEGRATION_ID,
    TENANT_ID,
    X_CUSTOM,
    W_INSERT_DT,
    W_UPDATE_DT,
    ETL_PROC_WID
  )
  VALUES (
    src.LEDGER_NAME,
    src.LEDGER_SHORT_NAME,
    src.LEDGER_DESC,
    src.CHART_OF_ACCOUNTS,
    src.CURRENCY_CODE,
    src.CALENDER_NAME,
    src.SLA_ACCOUNTING_METHOD_CODE,
    src.PERIOD_TYPE,
    src.LEDGER_CATEGORY_CODE,
    src.SET_ID,
    src.CREATED_BY_WID,
    src.CHANGED_BY_WID,
    src.CREATED_ON_DT,
    src.CHANGED_ON_DT,
    src.AUX1_CHANGED_ON_DT,
    src.AUX2_CHANGED_ON_DT,
    src.AUX3_CHANGED_ON_DT,
    src.AUX4_CHANGED_ON_DT,
    src.DELETE_FLG,
    src.DATASOURCE_NUM_ID,
    src.INTEGRATION_ID,
    src.TENANT_ID,
    src.X_CUSTOM,
    src.CURRENT_TS,
    src.CURRENT_TS,
    src.ETL_PROC_WID
  );

In [ ]:
%sql
-- 8) UPDATE CONTROL TABLE: Upsert W_ETL_LOAD_DATES
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT
    p.DATASOURCE_NUM_ID AS datasource_num_id,
    'SILOS_SIL_LEDGERDIMENSION' AS package_name,
    'W_LEDGER_D' AS target_table_name,
    p.ETL_USAGE_CODE AS etl_usage_code,
    p.ETL_PROC_WID AS etl_proc_wid,
    p.EXECUTION_ID AS load_plan_id,
    current_timestamp() AS etl_load_date,
    CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date
  FROM etl_params p
) src
ON tgt.datasource_num_id = src.datasource_num_id
  AND tgt.package_name = src.package_name
  AND tgt.etl_usage_code = src.etl_usage_code
WHEN MATCHED THEN
  UPDATE SET
    tgt.target_table_name = src.target_table_name,
    tgt.etl_proc_wid = src.etl_proc_wid,
    tgt.load_plan_id = src.load_plan_id,
    tgt.wip_load_start_date = src.wip_load_start_date,
    tgt.etl_load_date = src.etl_load_date,
    tgt.committed = src.committed
WHEN NOT MATCHED THEN
  INSERT (
    datasource_num_id, package_name, target_table_name, etl_usage_code,
    etl_proc_wid, load_plan_id, wip_load_start_date, last_max_date, etl_load_date, committed
  )
  VALUES (
    src.datasource_num_id, src.package_name, src.target_table_name, src.etl_usage_code,
    src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, NULL, src.etl_load_date, src.committed
  );

In [ ]:
%sql
-- 9) INSERT HISTORY LOG: Record ETL execution in history table
INSERT INTO workspace.oracle_edw.w_etl_load_dates_log
(
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
)
SELECT
        p.DATASOURCE_NUM_ID,
        'SILOS_SIL_LEDGERDIMENSION',
        'W_LEDGER_D',
        p.ETL_USAGE_CODE,
        p.ETL_PROC_WID,
        p.EXECUTION_ID,
        p.ETL_PROC_WID AS SESSION_ID,
        date_add(current_timestamp(), -p.PRUNE_DAYS) AS WIP_LOAD_START_DATE,
        NULL AS LAST_MAX_DATE,
        current_timestamp() AS ETL_LOAD_DATE,
        CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS COMMITTED
      FROM etl_params p;

In [ ]:
%sql
-- 10) CLEANUP: Drop temporary views
DROP VIEW IF EXISTS stg_ledger_with_action;
DROP VIEW IF EXISTS stg_ledger_transformed;
DROP VIEW IF EXISTS stg_ledger_with_lookups;
DROP VIEW IF EXISTS lkp_user_changed;
DROP VIEW IF EXISTS lkp_user_created;
DROP VIEW IF EXISTS etl_params;